In [1]:
import blazingsql
import cudf
import cuml

# Get some data!

In [2]:
!wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_00.csv -O ../data/taxi_00.csv
!wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_01.csv -O ../data/taxi_01.csv
!wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_02.csv -O ../data/taxi_02.csv
!wget https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_03.csv -O ../data/taxi_03.csv
!wget https://blazingsql-demos.s3-us-west-1.amazonaws.com/test.csv -O ../data/test.csv

--2019-10-24 08:52:13--  https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_00.csv
Resolving blazingsql-colab.s3.amazonaws.com (blazingsql-colab.s3.amazonaws.com)... 52.216.97.203
Connecting to blazingsql-colab.s3.amazonaws.com (blazingsql-colab.s3.amazonaws.com)|52.216.97.203|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 393974627 (376M) [application/x-www-form-urlencoded]
Saving to: ‘../data/taxi_00.csv’

../data/taxi_00.csv 100%[===================>] 375.72M  11.4MB/s    in 35s     

2019-10-24 08:52:49 (10.8 MB/s) - ‘../data/taxi_00.csv’ saved [393974627/393974627]

--2019-10-24 08:52:49--  https://blazingsql-colab.s3.amazonaws.com/taxi_data/taxi_01.csv
Resolving blazingsql-colab.s3.amazonaws.com (blazingsql-colab.s3.amazonaws.com)... 52.216.107.124
Connecting to blazingsql-colab.s3.amazonaws.com (blazingsql-colab.s3.amazonaws.com)|52.216.107.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 393961373 (376M) [application

# ETL

## Read and join *.csv files

In [2]:
import glob


names = ['key',
         'fare_amount',
         'pickup_longitude',
         'pickup_latitude',
         'dropoff_longitude',
         'dropoff_latitude',
         'passenger_count']

dtypes = ['date64',
          'float32',
          'float32',
          'float32',
          'float32',
          'float32',
          'float32']

_gdfs = []
for _filepath in glob.glob("../data/taxi_*.csv"): 
    _gdf = cudf.read_csv(_filepath, delimiter=',', dtype=dtypes, names=names)
    _gdfs.append(_gdf)

gdf = cudf.concat(_gdfs)

In [3]:
gdf.head()

,key,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2012-02-02 22:30:19.002,8.9,-73.988708,40.758804,-73.986519,40.737202,1.0
1,2014-09-20 07:19:24.001,4.0,-73.990204,40.746708,-73.994728,40.750515,1.0
2,2013-02-23 07:18:05.001,5.5,-74.016762,40.709438,-74.009003,40.719498,3.0
3,2015-04-18 23:49:27.009,13.5,-74.002708,40.733730,-73.986099,40.734776,1.0
4,2010-03-04 08:15:59.001,10.5,-73.988365,40.737663,-74.012459,40.713932,1.0


## Create a table

In [4]:
bc = blazingsql.BlazingContext()

BlazingContext ready


In [5]:
bc.create_table("taxi", gdf)

In [23]:
sql = """
SELECT 
  hour(key) AS hours, 
  month(key) AS months, 
  year(key) AS years, 
  dropoff_longitude - pickup_longitude AS longitude_distance, 
  dropoff_latitude - pickup_latitude AS latitude_distance, 
  passenger_count 
FROM 
  {}
"""
results = (bc.sql(sql.format("main.taxi"))
             .get())

In [25]:
(results.columns
        .head())

,$f0,$f1,$f2,$f3,$f4,passenger_count
0,22,2,2012,0.002190,-0.021603,1.0
1,7,9,2014,-0.004524,0.003807,1.0
2,7,2,2013,0.007759,0.010059,3.0
3,23,4,2015,0.016609,0.001045,1.0
4,8,3,2010,-0.024094,-0.023731,1.0


In [32]:
training_features_gdf = (results.columns
                                .fillna(0.0))

In [33]:
training_features_gdf.head()

,$f0,$f1,$f2,$f3,$f4,passenger_count
0,22,2,2012,0.002190,-0.021603,1.0
1,7,9,2014,-0.004524,0.003807,1.0
2,7,2,2013,0.007759,0.010059,3.0
3,23,4,2015,0.016609,0.001045,1.0
4,8,3,2010,-0.024094,-0.023731,1.0


In [34]:
_results = (bc.sql('SELECT fare_amount FROM main.taxi')
              .get())
training_target_gdf = (_results.columns
                               .fillna(0.0))

In [35]:
training_target_gdf.head()

,fare_amount
0,8.9
1,4.0
2,5.5
3,13.5
4,10.5


In [36]:
%%time
gpu_linear_regression = cuml.LinearRegression(fit_intercept=True, normalize=False, algorithm="eig")
gpu_linear_regression.fit(training_features_gdf, training_target_gdf)

CPU times: user 300 ms, sys: 86.6 ms, total: 387 ms
Wall time: 385 ms


LinearRegression(algorithm='eig', fit_intercept=True, normalize=False, handle=<cuml.common.handle.Handle object at 0x14f570f1a480>)

In [37]:
gpu_linear_regression.coef_

0   -0.027298
1    0.102763
2    0.576596
3    0.000888
4   -0.000524
5    0.092905
dtype: float32

In [38]:
gpu_linear_regression.intercept_

-1149.3880615234375

In [39]:
_gdf = cudf.read_csv("../data/test.csv", delimiter= ',', dtype=dtypes, names=names)
bc.create_table('test', _gdf)

In [41]:
#Query Test Data Table to Create GDF
_results = (bc.sql(sql.format("main.test"))
              .get())
testing_features_gdf = (_results.columns
                                .fillna(0.0))

In [42]:
testing_features_gdf.head()

,$f0,$f1,$f2,$f3,$f4,passenger_count
0,13,1,2015,-0.008110,-0.019970,1.0
1,13,1,2015,-0.012024,0.019814,1.0
2,11,10,2011,0.002869,-0.005119,1.0
3,21,12,2012,-0.009277,-0.016178,1.0
4,21,12,2012,-0.022537,-0.045345,1.0


In [46]:
_predictions = gpu_linear_regression.predict(testing_features_gdf)
with_predictions_gdf = testing_features_gdf.assign(predicted_fare=_predictions)

In [47]:
with_predictions_gdf.head()

,$f0,$f1,$f2,$f3,$f4,passenger_count,predicted_fare
0,13,1,2015,-0.008110,-0.019970,1.0,12.293823
1,13,1,2015,-0.012024,0.019814,1.0,12.293823
2,11,10,2011,0.002869,-0.005119,1.0,10.966919
3,21,12,2012,-0.009277,-0.016178,1.0,11.476074
4,21,12,2012,-0.022537,-0.045345,1.0,11.476074
